In [3]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.7/239.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 3

In [4]:
pip install datasets

In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
import csv
import warnings
from typing import Any, Dict, Tuple
import re
from transformers import AutoTokenizer, pipeline


warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

# Dataset

In [6]:
from datasets import load_dataset

#load the test data for the generation step for all language pairs
data = load_dataset("Eloquent/HalluciGen-Translation", "test_generation")


AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'

In [ ]:
data

In [ ]:
#load model Llama2
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## Prompt 2


*   Give model a role
*   Chain-of-Thought approach: examples to illustrate the task



In [ ]:
df_de_en = data["test_generation_de_en"].to_pandas()
df_de_en.head()

,id,langpair,source
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ..."


In [ ]:
def generate_translation(source, target_language, example):
  answer_format = {"hyp+": "", "hyp-": ""}

  # prompt for translation generation task
  user_prompt = f'''
  As an AI model, your task is to generate two translation hypothesis given the 'src' below.
  The first translation labelled as 'hyp+' should be supported by 'src' and the second translation labelled as 'hyp-' should not be supported by 'src'.
  Provide the result in the following format: {answer_format}. Both hypotheses should be translated into {target_language}.

  Here is an example to illustrate this:
  {example}

  Src: {source}

  Result:
  '''
  messages = [
      {"role": "system", "content": f"You are a text generator. You are specialized in translating texts to {target_language}"},
      {"role": "user", "content": user_prompt},
    ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=250,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
  generated_paraphrase = outputs[0]["generated_text"][len(prompt):]

  return generated_paraphrase

In [ ]:
def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    print(result_text)
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    return result_text

In [ ]:
source = "Ich gehe morgen in die Schule, weil ich Sportunterricht habe."
result = generate_translation(source, "English", example)
extract_json_data(result)

{'hyp+': 'I will go to school tomorrow because I have physical education class.', 'hyp-': 'I will not go to school tomorrow because I don't have any classes.'}


JSONDecodeError: Expecting ',' delimiter: line 1 column 139 (char 138)

## Generate dataset

In [ ]:

import os
import numpy as np
import pandas as pd

def save_results(target_language, df, csv_filename, example):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)
    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']

            try:
                # Generate paraphrase
                result = generate_translation(source, target_language, example)
                #print("source: ", source)
                extracted_hyp = extract_json_data(result)

                hyp_support = extracted_hyp['hyp+']
                #print("hyp+: ", hyp_support)
                hyp_hallu = extracted_hyp['hyp-']
                #print("hyp-: ", hyp_hallu)

                # Update DataFrame with new values
                chunk.at[index, 'hyp+'] = hyp_support
                chunk.at[index, 'hyp-'] = hyp_hallu
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
            print("finished chunk")
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)
            print("finished chunk")

        # Load CSV file
        df = pd.read_csv(csv_filename)
        display(df)
    return skipped_rows

# German-English

In [ ]:
df_de_en = data["test_generation_de_en"].to_pandas()
df_de_en.head()

,id,langpair,source
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ..."


In [ ]:
#load the trial data only for the German->English pair
trial_ds_de_en = load_dataset("Eloquent/HalluciGen-Translation", name="trial", split="trial_de_en")

Generating trial_de_en split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_en_de split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_fr_en split:   0%|          | 0/10 [00:00<?, ? examples/s]

Generating trial_en_fr split:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
# load trial dataset to get one example for chain-of-thought approach
trial_de_en = trial_ds_de_en.to_pandas()

In [ ]:
trial_de_en.head()

,id,langpair,source,hyp1,hyp2,type,label
0,0,de-en,In der Phase marschieren Wanderameisen bei Nac...,"In the phase, wandering ants march at night an...","In the nomadic phase, wandering ants march at ...",addition,hyp2
1,1,de-en,Nachdem sich der Frontsänger Steven Tyler währ...,After front singer Steven Tyler injured himsel...,After front singer Steven Tyler injured himsel...,date,hyp1
2,2,de-en,Der Wirtschaftsprüfer hat die Entwicklerin bei...,The auditor caught the male developer cheating.,The auditor caught the female developer cheating.,gender,hyp1
3,3,de-en,Er wurde in aufeinanderfolgenden Coups durch O...,He was replaced in successive coups by Olusegu...,He was replaced by Olusegun Obasanjo (in 1975)...,named-entity,hyp1
4,4,de-en,Der NVIDIA TITAN V wurde von Nvidia am 7. Deze...,"On December 7, 2017 NVIDIA officially announce...",The NVIDIA TITAN V was officially announced by...,number,hyp2


In [ ]:
# create example for user prompt
example = f'''src: {trial_de_en.iloc[0]['source']}
  hyp+: {trial_de_en.iloc[0]['hyp1']}
  This is a translation that is supported by the source. It is the exact translation of 'src'.
  hyp-: {trial_de_en.iloc[0]['hyp2']}
  This is a translation, which is not supported by the source. It includes an addition, which is "nomadic" in this case. In 'src', there is no mention about a 'nomadic' phase. Hence, it is labelled as 'hyp-'.'''
print(example)

src: In der Phase marschieren Wanderameisen bei Nacht und machen tagsüber Rast.
  hyp+: In the phase, wandering ants march at night and rest during the day.
  This is a translation that is supported by the source. It is the exact translation of 'src'.
  hyp-: In the nomadic phase, wandering ants march at night and rest during the day.
  This is a translation, which is not supported by the source. It includes an addition, which is "nomadic" in this case. In 'src', there is no mention about a 'nomadic' phase. Hence, it is labelled as 'hyp-'.


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama2/prompt2/results_llama2_de_eng.csv"
skipped_rows = save_results("English", df_de_en, csv_filename, example)

{'hyp+': 'In countries, offering luxury guesthouses has today become an art form elevated to a new level.', 'hyp-': 'In nations, the provision of luxury guesthouses has been elevated to an art form, with a new level of sophistication.'}
{'hyp+': 'In Britain, this is a common practice, but in Scotland, the judiciary operates differently, and courts have deemed the publication of photos as potentially harmful.', 'hyp-': 'In Scotland, this is a standard practice, but the judiciary operates differently, and courts have deemed the publication of photos as potentially dangerous.'}
{'hyp+': 'On the basis of this fossil, it can be said that the separation occurred much earlier than expected based on evidence',}
Skipping row 2 due to error in data extraction.
{'hyp+': 'In the phase of marching, wandering ants move at night and rest during the day.', 'hyp-': 'In the never-ending quest for adventure, we will never overtake a car, build a fountain in our backyard, travel to explore ancient ruins, 

,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...","In nations, the provision of luxury guesthouse..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","In Britain, this is a common practice, but in ...","In Scotland, this is a standard practice, but ..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,NaN,NaN
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","In the phase of marching, wandering ants move ...","In the never-ending quest for adventure, we wi..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...",The Avenger-class mine defense ship was on its...,The ship was on a journey to an unknown destin...
5,5,de-en,Die Spitze war ein spezielles Heiligtum für de...,The tip was a special shrine dedicated to the ...,"In the peak, a unique holy site was reserved f..."
6,6,de-en,Kaltes Wetter ist vielleicht die einzige Gefah...,"In cold weather, perhaps the only danger that ...",Cold weather is perhaps the only threat that u...
7,7,de-en,Das Fliegen in der Nähe eines Flughafens oder ...,In the vicinity of an airport or over a large ...,In the vicinity of an airport or over a large ...
8,8,de-en,"gab die US Navy an, sie sei dabei, den Zwisch...",The US Navy announced that they are investigat...,"In a surprising turn of events, the US Navy re..."
9,9,de-en,In den letzten 20 Jahren ist die Auswahl in Up...,NaN,NaN


{'hyp+': 'In the phase of returning, Perry plans to evaluate the results of the preliminary election tonight and decide if there is a further path for him in the race.', 'hyp-': 'Perry stated that he will return to Texas to assess the outcome of the preliminary election tonight and decide if he will continue in the race, but later changed his mind and decided to stay in the race and compete in the preliminary elections in South Carolina on January 21.'}
{'hyp+': 'Huhn and Pryce are due to appear before the Westminster Magistrates Court on February 16th.', 'hyp-': 'Huhn and Pryce are expected to appear before the Westminster Magistrates Court on February 16th, but the source text does not provide any information about the reason for their appearance.'}
{'hyp+': 'Bachmann, who won the Ames Straw Poll in August, decided to end her campaign.', 'hyp-': 'Bachmann, who won the Ames Straw Poll in August, decided to continue her campaign, despite the challenge from her opponents'}
{'hyp+': 'In 

,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...","In nations, the provision of luxury guesthouse..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","In Britain, this is a common practice, but in ...","In Scotland, this is a standard practice, but ..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,NaN,NaN
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","In the phase of marching, wandering ants move ...","In the never-ending quest for adventure, we wi..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...",The Avenger-class mine defense ship was on its...,The ship was on a journey to an unknown destin...
5,5,de-en,Die Spitze war ein spezielles Heiligtum für de...,The tip was a special shrine dedicated to the ...,"In the peak, a unique holy site was reserved f..."
6,6,de-en,Kaltes Wetter ist vielleicht die einzige Gefah...,"In cold weather, perhaps the only danger that ...",Cold weather is perhaps the only threat that u...
7,7,de-en,Das Fliegen in der Nähe eines Flughafens oder ...,In the vicinity of an airport or over a large ...,In the vicinity of an airport or over a large ...
8,8,de-en,"gab die US Navy an, sie sei dabei, den Zwisch...",The US Navy announced that they are investigat...,"In a surprising turn of events, the US Navy re..."
9,9,de-en,In den letzten 20 Jahren ist die Auswahl in Up...,NaN,NaN


{'hyp+': 'The farmer walked faster than the tailor.', 'hyp-': 'The farmer loved faster than the tailor.'}
{'hyp+': 'The doctor worried about the hairdresser.', 'hyp-': 'The doctor concerned himself with the hairdresser.'}
{'hyp+': 'The mechanic called the accountant.', 'hyp-': 'The mechanic yelled at the accountant.'}
{'hyp+': 'The driver asked the nurse for a check-up.', 'hyp-': 'The driver asked the nurse to perform a routine examination.'}
{'hyp+': 'The caretaker equipped the writer with new tools.', 'hyp-': 'In the caretaker's phase, the writer was provided with new tools.'}
Skipping row 24 due to error in data extraction.
{'hyp+': 'The janitor fell in love with the secretary.', 'hyp-': 'The janitor became enamored with the secretary, but their relationship was forbidden'}
{'hyp+': 'The worker dug a hole for the secretary.', 'hyp-': 'The worker dug a hole for the boss.'}
{'hyp+': 'The designer wanted to impress the salesperson.', 'hyp-': 'The designer wanted to intimidate the sales

...

,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...","In nations, the provision of luxury guesthouse..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","In Britain, this is a common practice, but in ...","In Scotland, this is a standard practice, but ..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,NaN,NaN
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","In the phase of marching, wandering ants move ...","In the never-ending quest for adventure, we wi..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...",The Avenger-class mine defense ship was on its...,The ship was on a journey to an unknown destin...
...,...,...,...,...,...
75,75,de-en,Lowther starb 1872 und da er keine legitimen E...,"In the phase of marching, the ants walk at nig...",Lowther died in 1872 and since he had no legit...
76,76,de-en,Hinweis: Nicht alle der oben genannten Details...,"In the phase of marching, ants wander at night...","In a nomadic state, ants march at night and re..."
77,77,de-en,"Diese Art ist in der Schweiz, in Frankreich un...",NaN,NaN
78,78,de-en,Die armenische nationale Befreiungsbewegung wa...,The Armenian national liberation movement was ...,"In the aftermath of the war, the Armenian nati..."


{'hyp+': 'In the genus Psilochilus, ants march at night and rest during the day.', 'hyp-': 'In the native range of Psilochilus, ants are nomadic and roam freely during the day.'}
{'hyp+': 'The Cheia River is a tributary of the Silia River in Romania.', 'hyp-': 'In the river, the water is always flowing rapidly due to the steep slope of the surrounding hills.'}
{'hyp+': 'Alton lies above the confluence of the Missouri River and Mississippi River.', 'hyp-': 'In Alton, the Mississippi River flows into the Missouri River'}
{'hyp+': 'In this interpretation of the law, a strong emphasis is placed on the literal wording of the law.', 'hyp-': 'This interpretation of the law focuses on the literal meaning of the words, rather than their broader context.'}
{'hyp+': 'In the phase of teaching, Jürgen Reichen, a Swiss teacher (progressive education), established in 1982 this "Writing to Read" method.', 'hyp-': 'In the early years of his teaching career, Jürgen Reichen, a Swiss teacher (progressive 

,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...","In nations, the provision of luxury guesthouse..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","In Britain, this is a common practice, but in ...","In Scotland, this is a standard practice, but ..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,NaN,NaN
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","In the phase of marching, wandering ants move ...","In the never-ending quest for adventure, we wi..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...",The Avenger-class mine defense ship was on its...,The ship was on a journey to an unknown destin...
...,...,...,...,...,...
85,85,de-en,Wenn `` R '' die zugehörige Äquivalenzrelation...,"In the phase of marching, ants wander at night...",In the absence of a corresponding equivalence ...
86,86,de-en,"Es zeigt 362 verschiedene alte Holzarten, Büsc...","In the phase of marching, wandering ants march...","In the exhibit, it shows 362 different types o..."
87,87,de-en,"Obrovac ist eine Stadt in Norddalmatien, in Kr...","In the city of Obrovac, located in the norther...","In the town of Obrovac, situated in the rural ..."
88,88,de-en,Danach wechselte er 1986 zu The Star und verli...,"After that, he switched to The Star in 1986 an...","In 1986, he moved to The Star and left The Mus..."


{'hyp+': 'During his participation in the DTM, the Audi V8 was in competition with the smaller and lighter Mercedes 190, BMW M3, and the slightly smaller Opel Omega 3000.', 'hyp-': 'In the DTM, the Audi V8 competed against the smaller and lighter Mercedes 190, BMW M3, and the slightly smaller Opel Omega 3000, despite being larger and heavier than them.'}
{'hyp+': 'Robert Vincent Goldsborough was born on October 3, 1937, in Chicago, Illinois, as the son of architect Robert Goldsborough and Wilma (Janak) Goldsborough.', 'hyp-': 'In the 1930s, Robert Vincent Goldsborough was born in Chicago as the son of a wealthy architect and his wife.'}
{'hyp+': 'Taylor remained active as a scout for the Chicago White Sox and the Milwaukee and Atlanta Braves until his death in baseball.', 'hyp-': 'Taylor was a scout for the Chicago White Sox and the Milwaukee and Atlanta Braves until his death, but he never played professional baseball himself.'}
{'hyp+': 'In the aftermath of their activation, the 99th

,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...","In nations, the provision of luxury guesthouse..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","In Britain, this is a common practice, but in ...","In Scotland, this is a standard practice, but ..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,NaN,NaN
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","In the phase of marching, wandering ants move ...","In the never-ending quest for adventure, we wi..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...",The Avenger-class mine defense ship was on its...,The ship was on a journey to an unknown destin...
...,...,...,...,...,...
95,95,de-en,Die Anzahl der gemeldeten Brände betrug Anfang...,"In the early stages of February, the number of...","In the beginning of February, the number of re..."
96,96,de-en,Der Ponoru ist eine Nebenfluss des Horezu in R...,"In the night, the ants march during the phase ...",The Ponoru is a tributary of the Horezu in Rom...
97,97,de-en,Leslie Waddington eröffnete 1966 seine eigene ...,"In 1966, Leslie Waddington opened his own gall...","In the 1960s, Leslie Waddington launched his o..."
98,98,de-en,In der Zwischenzeit kaufte die Air Force die S...,"In the interim, the Air Force purchased the SR...","In the meantime, the Air Force acquired the SR..."


In [ ]:
skipped_rows

[2, 9, 13, 17, 24, 31, 41, 44, 51, 52, 57, 59, 60, 67, 70, 73, 77, 79, 84, 99]

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama2/prompt2/results_llama2_de_eng.csv"
df_results = pd.read_csv(csv_filename)
len(df_results)

100

In [ ]:

def fix_skipped_rows(target_language, df, csv_filename, skipped_rows, example):
    skipped_rows2 = []
    for index in skipped_rows:
        row = df.loc[index]

        source = row['source']

        try:
            result = generate_translation(source, target_language,example)
            # Find the start and end indices of the JSON data
            start_index = result.find('{')
            end_index = result.find('}') + 1
            result_text = result[start_index:end_index]
            result_text = result_text.replace('"', "'").replace("{'hyp+': '", '{"hyp+": "').replace(".', 'hyp-': '", '.", "hyp-": "').replace(".',\n'hyp-': '", '.",\n"hyp-": "').replace(".', \n'hyp-': '", '.", \n"hyp-": "').replace("'}", '"}')
            print("result text ", result_text)
            #print("result text ", result_text)
            extracted_hyp = json.loads(result_text)
            df.at[index, 'hyp+'] =  extracted_hyp['hyp+']
            df.at[index, 'hyp-'] =  extracted_hyp['hyp-']
        except Exception as e:
            print(f"Skipping row {index} due to error in data extraction.")
            skipped_rows2.append(index)
            continue

    df.to_csv(csv_filename, index=False)
    return skipped_rows2


In [ ]:
skipped_rows = fix_skipped_rows("English", df_results, "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama2/prompt2/results_llama2_de_eng_fixed.csv", skipped_rows, example)

result text  {"hyp+": "On the basis of this fossil, it can be said that the separation occurred much earlier than expected, based on evidence.", "hyp-": "In the fossil record, it appears that the separation occurred much earlier than previously believed, with no mention of a specific timeframe."}
result text  {"hyp+": "In the last 20 years, the selection in Uptown Charlotte has exponentially grown.", "hyp-": "In Uptown Charlotte, the selection has grown exponentially over the last 20 years, but the sentence is not grammatically correct and does not make sense"}
result text  {"hyp+": "In the aftermath of the real estate fraud, a meeting was held on March 20th at the East Bay Community Law Center, where numerous families sought legal aid to resist evictions.", "hyp-": "For the victims of real estate fraud, a meeting was held on March 20th at the East Bay Community Law Center, where they sought legal aid to fight against evictions, but the source text does not provide any information abou

In [ ]:
skipped_rows

[41, 52, 77, 99]

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama2/prompt2/results_llama2_de_eng_fixed.csv")
df.head()

,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...","In nations, the provision of luxury guesthouse..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","In Britain, this is a common practice, but in ...","In Scotland, this is a standard practice, but ..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"On the basis of this fossil, it can be said th...","In the fossil record, it appears that the sepa..."
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","In the phase of marching, wandering ants move ...","In the never-ending quest for adventure, we wi..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...",The Avenger-class mine defense ship was on its...,The ship was on a journey to an unknown destin...


In [ ]:
translation = generate_translation(df.iloc[3]["source"], "English", example)

In [ ]:
df.iloc[3]["source"]

'Wir werden niemals ein Auto überholen, einen Brunnen in unserem Hinterhof bauen, reisen, um antike Ruinen zu untersuchen, oder das Haus unseres Nachbarn umgestalten.'

In [ ]:
print(translation)

  Sure, I'd be happy to help! Here are two translation hypotheses for the given source text:

{'hyp+': 'We will never overtake a car, build a well in our backyard, travel to study ancient ruins, or renovate our neighbor's house.'}

{'hyp-': 'We will never catch up with a car, construct a well in our backyard, travel to explore ancient ruins, or remodel our neighbor's home.'}

In the first hypothesis, the translation is supported by the source text, and it provides a clear and concise translation of the original sentence. In the second hypothesis, the translation is not supported by the source text, and it introduces an additional element ("nomadic") that is not present in the original sentence.


In [ ]:
translation = generate_translation(df.iloc[16]["source"], "English", example)
print(translation)

  Sure, I'd be happy to help! Here are two translation hypotheses for the given source text:

{'hyp+': 'In the phase of marching, ants walk at night and rest during the day.', 'hyp-': 'In the construction phase, the height of the underpass is 15 meters. The construction was completed in August 2011, but the opening for traffic did not occur until March 2017.'}

Explanation:

The source text describes the behavior of ants marching at night and resting during the day. This is a direct translation of the original German text, and is therefore labelled as 'hyp+'.

The second hypothesis includes additional information not present in the source text, specifically the height of the underpass and the dates of construction and opening for traffic. This is labelled as 'hyp-' because it is not supported by the source.


In [ ]:
## Llama2 seems to have difficulties when using few shot examples, so we are leaving it as it is..